In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

A40
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack


In [4]:
# import torch
# from torch.utils import data
# import numpy as np
# from os.path import join as pjoin
# import random
# import codecs as cs
# from tqdm import tqdm



# class VQMotionDataset(data.Dataset):
#     def __init__(self, dataset_name, window_size = 64, unit_length = 4):
#         self.window_size = window_size
#         self.unit_length = unit_length
#         self.dataset_name = dataset_name

#         if dataset_name == 't2m':
#             self.data_root = '/srv/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D'
#             self.motion_dir = pjoin(self.data_root, 'new_joint_vecs')
#             self.text_dir = pjoin(self.data_root, 'texts')
#             self.joints_num = 22
#             self.max_motion_length = 196
#             self.meta_dir = ''

#         elif dataset_name == 'kit':
#             self.data_root = './dataset/KIT-ML'
#             self.motion_dir = pjoin(self.data_root, 'new_joint_vecs')
#             self.text_dir = pjoin(self.data_root, 'texts')
#             self.joints_num = 21

#             self.max_motion_length = 196
#             self.meta_dir = 'checkpoints/kit/VQVAEV3_CB1024_CMT_H1024_NRES3/meta'
        
#         joints_num = self.joints_num

#         mean = np.load(pjoin(self.data_root, 'Mean.npy'))
#         std = np.load(pjoin(self.data_root, 'Std.npy'))

#         split_file = pjoin(self.data_root, 'val.txt')

#         self.data = []
#         self.lengths = []
#         id_list = []
#         with cs.open(split_file, 'r') as f:
#             for line in f.readlines():
#                 id_list.append(line.strip())

#         for name in tqdm(id_list):
#             try:
#                 motion = np.load(pjoin(self.motion_dir, name + '.npy'))
#                 if motion.shape[0] < self.window_size:
#                     continue
#                 self.lengths.append(motion.shape[0] - self.window_size)
#                 self.data.append(motion)
#             except:
#                 # Some motion may not exist in KIT dataset
#                 pass

            
#         self.mean = mean
#         self.std = std
#         print("Total number of motions {}".format(len(self.data)))

#     def inv_transform(self, data):
#         return data * self.std + self.mean
    
#     def compute_sampling_prob(self) : 
        
#         prob = np.array(self.lengths, dtype=np.float32)
#         prob /= np.sum(prob)
#         return prob
    
#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, item):
#         motion = self.data[item]
        
#         idx = random.randint(0, len(motion) - self.window_size)

#         motion = motion[idx:idx+self.window_size]
#         "Z Normalization"
#         motion = (motion - self.mean) / self.std

#         return motion

In [523]:
trainSet = VQMotionDataset("aist", data_root="/srv/scratch/sanisetty3/music_motion/AIST")

100%|██████████| 952/952 [00:54<00:00, 17.47it/s]

Total number of motions 951


In [24]:
def DATALoader(
            dataset,
            batch_size,
            num_workers = 0,
            shuffle = True,
           ):
    train_loader = torch.utils.data.DataLoader(dataset,
                                                batch_size,
                                                shuffle=shuffle,
                                                #sampler=sampler,
                                                num_workers=num_workers,
                                                #collate_fn=collate_fn,
                                                drop_last = True)

    return train_loader

In [25]:
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

In [53]:
train_loader = DATALoader(trainSet , 2)
train_loader_iter = cycle(train_loader)

In [54]:
gt_motion = next(train_loader_iter)

In [55]:
gt_motion.shape

torch.Size([2, 40, 263])

In [29]:
from core.models.vqvae import VQMotionModel
from core.models.loss import ReConsLoss


In [30]:
loss_fnc = ReConsLoss("l2", 22)

In [31]:
model = VQMotionModel()

VectorQuantize(
  (project_in): Identity()
  (project_out): Identity()
  (_codebook): EuclideanCodebook()
)

In [32]:
pred_motion , indices, commit_loss = model(gt_motion)

In [33]:
loss_motion = loss_fnc(pred_motion, gt_motion)


In [23]:
loss_vel = loss_fnc.forward_vel(pred_motion, gt_motion)

In [537]:
from utils.motion_process import recover_from_ric
import visualize.plot_3d_global as plot_3d
from glob import glob

In [536]:
def to_xyz(motion):
    motion_xyz = recover_from_ric(motion.cpu().float()*trainSet.std+trainSet.mean, 22)
    motion_xyz = motion_xyz.reshape(motion.shape[0],-1, 22, 3)
    return motion_xyz

            
def sample_render(motion_xyz , name , save_path):
    print(f"render start")
    
    gt_pose_vis = plot_3d.draw_to_batch(motion_xyz.numpy(),None, [os.path.join(save_path,name + ".gif")])



In [37]:
gt_motion.shape

torch.Size([8, 60, 263])

In [40]:
sample_render(to_xyz(pred_motion[0:1].detach().cpu()), "aist0_pred" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals")

render start


## Variable motion lengths


In [5]:
import torch
from torch.utils import data
import numpy as np
from os.path import join as pjoin
import random
import codecs as cs
from tqdm import tqdm

In [330]:

class MotionCollator():
    def __init__(self, max_seq_length):
        self.max_seq_length = max_seq_length
        self.bos = torch.LongTensor(([0]))
        self.eos = torch.LongTensor(([2]))
        self.pad = torch.LongTensor(([1]))

    def __call__(self, samples):
        

        pad_batch_inputs = []
        pad_batch_mask = []
        motion_lengths = []
        names = []
        max_len = max([sample.shape[0] for sample, name in samples])


        for inp,name in samples:
            n,d = inp.shape
            diff = max_len - n
            mask = torch.BoolTensor([1]*n + [0]*diff)
            padded = torch.concatenate((torch.tensor(inp) , torch.ones((diff,d))*self.pad))
            pad_batch_inputs.append(padded)
            pad_batch_mask.append(mask)
            motion_lengths.append(n)
            names.append(name)

    
        batch = {
            "motion": torch.stack(pad_batch_inputs , 0),
            "motion_length": torch.Tensor(motion_lengths),
            "motion_mask" : torch.stack(pad_batch_mask , 0),
            "names" : np.array(names)

        }

   
        return batch    
    

In [6]:
from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from core.datasets import vqa_motion_dataset

In [7]:
# train_ds = vqa_motion_dataset.VQMotionDataset("t2m",split = "render", data_root = "/srv/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D")


100%|██████████| 10/10 [00:00<00:00, 864.13it/s]

Total number of motions 8


In [8]:
train_ds = VQVarLenMotionDataset("t2m", split = "render" , data_root = "/srv/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D")


changing range to: 60 - 60


100%|██████████| 10/10 [00:00<00:00, 1092.98it/s]

Total number of motions 8


In [9]:
collate_fn = MotionCollator(200)

In [10]:
train_loader = DATALoader(train_ds,
                                                4,
                                                #sampler=sampler,
                                                collate_fn=collate_fn,
                                                )

In [11]:
train_loader.batch_size

4

In [48]:
for batch in train_loader:
    break

In [49]:
gt_motion = batch["motion"]
print(gt_motion.shape)
print(batch["motion_lengths"])

torch.Size([4, 92, 263])
tensor([81., 82., 68., 92.])


In [50]:
train_loader.dataset.set_stage(3)

changing range to: 60 - 140


In [118]:
motion_len = int(batch.get("motion_lengths" , [gt_motion.shape[1]])[0])

In [119]:
gt_motion[:,:motion_len,:].shape

torch.Size([1, 97, 263])

In [17]:
batch["motion"].shape

torch.Size([4, 192, 263])

In [43]:
from core.models.vqvae import MotionTransformer, MotionDecoder, LinearEmbedding, VQMotionModel
from core.quantization.core_vq import VectorQuantization
from x_transformers.x_transformers import AttentionLayers, Encoder, Decoder, exists, default, always,ScaledSinusoidalEmbedding,AbsolutePositionalEmbedding, l2norm
from core.models.loss import ReConsLoss


In [39]:
from configs.config import cfg
model = VQMotionModel(cfg.vqvae).cuda()

In [37]:
total = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total training params: %.2fM" % (total / 1e6))

Total training params: 151.57M


In [40]:
pred_motion , indices, commit_loss = model(batch["motion"].cuda() , mask = batch["motion_mask"].cuda())

/coc/scratch/sanisetty3/music_motion/motion_vqvae/core/quantization/core_vq.py:375: UserWarning: When using RVQ in training model, first check https://github.com/facebookresearch/encodec/issues/25 . The bug wasn't fixed here for reproducibility.
  warnings.warn('When using RVQ in training model, first check '


In [41]:
pred_motion.shape

torch.Size([4, 192, 263])

In [44]:
loss_fnc = ReConsLoss("l1", 22)

In [45]:
loss_fnc(pred_motion, batch["motion"].cuda() ,  batch["motion_mask"].cuda())

tensor(1.1719, device='cuda:0', grad_fn=<MulBackward0>)

In [47]:
loss_fnc.forward_vel(pred_motion, batch["motion"].cuda() ,  batch["motion_mask"].cuda())

tensor(0.2821, device='cuda:0', grad_fn=<MulBackward0>)

In [338]:
motionEncoder = MotionTransformer(
        inp_dim =263,
        max_seq_len = 200,
        scaled_sinu_pos_emb = True,
        attn_layers = Encoder(
            dim = 768,
            depth = 2,
            heads = 4,

        )
    )

In [345]:
encoded_motion = motionEncoder(batch["motion"] , mask = batch["motion_mask"])
print(encoded_motion.shape)

torch.Size([4, 107, 768])


In [347]:
vq = VectorQuantization(
        dim = 768,
        codebook_dim = 128,
        codebook_size = 1024,
        decay = 0.95,
        commitment_weight = 1,
        kmeans_init = True,
        threshold_ema_dead_code = 2,
)

In [348]:
quantized_enc_motion, indices, commit_loss = vq(encoded_motion)
print(quantized_enc_motion.shape)

torch.Size([4, 107, 768])


/coc/scratch/sanisetty3/music_motion/motion_vqvae/core/quantization/core_vq.py:350: UserWarning: When using RVQ in training model, first check https://github.com/facebookresearch/encodec/issues/25 . The bug wasn't fixed here for reproducibility.
  warnings.warn('When using RVQ in training model, first check '


In [358]:
embed = vq.codebook.t()
embed.pow(2).sum(0, keepdim=True).shape

torch.Size([1, 1024])

In [410]:
x = encoded_motion[:,:,:128]
shape = x.shape

In [411]:
x = rearrange(x, "... d -> (...) d")
x

tensor([[-0.3801,  0.7024,  1.5966,  ...,  0.7535, -1.5004, -0.4283],
        [-0.2908,  0.5800,  1.6096,  ...,  0.4755, -1.5223, -0.3614],
        [-0.2561,  0.4115,  1.5109,  ...,  0.1626, -1.4867, -0.2438],
        ...,
        [ 0.5579,  0.1785,  0.0483,  ..., -0.8444, -0.6883,  1.2005],
        [ 0.5358,  0.1656,  0.1023,  ..., -0.8245, -0.6591,  1.2180],
        [ 0.5378,  0.0897,  0.1818,  ..., -0.8273, -0.6676,  1.2187]],
       grad_fn=<ReshapeAliasBackward0>)

In [412]:
dist = -(
    x.pow(2).sum(1, keepdim=True)
    - 2 * x @ embed
    + embed.pow(2).sum(0, keepdim=True)
)


In [413]:
dist.shape

torch.Size([428, 1024])

In [414]:
embed_ind = dist.max(dim=-1).indices

In [415]:
embed_ind.shape

torch.Size([428])

In [416]:
inp_mask = rearrange(batch["motion_mask"], "... -> (...)")
inp_mask.shape


torch.Size([428])

In [417]:
mask_value = -torch.finfo(dist.dtype).max

In [440]:
dist[106]

tensor([-1.6291e+02, -1.6781e+02, -1.2629e+02,  ..., -3.8359e+11,
        -3.0852e+11, -4.7003e+11], grad_fn=<SelectBackward0>)

In [418]:
dist2 = (dist + dist.masked_fill(~inp_mask[...,None] , mask_value))

In [419]:
embed_ind2 = dist2.max(dim=-1).indices

In [435]:
embed_ind2[105]

tensor(0)

In [436]:
embed_onehot = F.one_hot(embed_ind2, 1024)

In [424]:
embed_ind = vq._codebook.postprocess_emb(embed_ind, shape)
quantize = vq._codebook.dequantize(embed_ind)

In [446]:
embed_onehot.shape

torch.Size([428, 1024])

In [445]:
embed_onehot.sum(0).shape

torch.Size([1024])

In [464]:
x.shape

torch.Size([428, 128])

In [459]:
embed_onehot2 = (embed_onehot * inp_mask[...,None] ).type(x.dtype)

In [460]:
embed_onehot2.shape

torch.Size([428, 1024])

In [463]:
(x.T@embed_onehot2)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<MmBackward0>)

In [466]:
encoded_motion.shape

torch.Size([4, 107, 768])

In [465]:
batch["motion_mask"].shape

torch.Size([4, 107])

In [524]:
quantized_enc_motion, indices, commit_loss = vq(encoded_motion , batch["motion_mask"])
print(indices.shape)

torch.Size([4, 107])


/coc/scratch/sanisetty3/music_motion/motion_vqvae/core/quantization/core_vq.py:375: UserWarning: When using RVQ in training model, first check https://github.com/facebookresearch/encodec/issues/25 . The bug wasn't fixed here for reproducibility.
  warnings.warn('When using RVQ in training model, first check '


In [475]:
motionDecoder = MotionDecoder(
    dim = 768,
    logit_dim = 263,
    attn_layers = Decoder(
            dim = 768,
            depth = 2,
            heads = 4,
        )
    )

In [480]:
pred_motion = motionDecoder(quantized_enc_motion, inp_mask = batch["motion_mask"])

In [481]:
pred_motion.shape

torch.Size([4, 107, 263])

In [483]:
pred_motion[batch["motion_mask"]].shape

torch.Size([380, 263])

In [484]:
batch["motion"].shape

torch.Size([4, 107, 263])

In [503]:
F.mse_loss(batch["motion"] * batch["motion_mask"][...,None] , pred_motion*batch["motion"] * batch["motion_mask"][...,None], reduction = "sum")

tensor(88422.6406, grad_fn=<MseLossBackward0>)

In [ ]:
0.7855

In [497]:
batch["motion"] .numel()/(batch["motion_mask"].sum()*263)

tensor(1.1263)

In [493]:
batch["motion"].numel()

112564

In [495]:
batch["motion_mask"].shape

torch.Size([4, 107])

In [496]:
batch["motion_mask"].sum()*263

tensor(99940)

## Decode test

In [5]:
class Dict2Class(object):
      
    def __init__(self, my_dict):
          
        for key in my_dict:
            setattr(self, key, my_dict[key])

In [6]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--data_folder', default="/srv/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D", help="folder with train and test data")
parser.add_argument('--pretrained', default='')
parser.add_argument('--resume', default=True, type = bool)
parser.add_argument('--output_dir', default="./checkpoints/vq_768_128")
parser.add_argument('--evaluate', action='store_true')
parser.add_argument('--seed', default=42, type=int)
parser.add_argument('--fp16', default=True, type=bool)
parser.add_argument("--dataset_name", type=str, default='aist', help="t2m or kit or aist")
parser.add_argument('--var_len', default=False, type=bool)


parser.add_argument('--train_bs', default=64, type=int,)
parser.add_argument('--eval_bs', default=64, type=int,)
parser.add_argument('--gradient_accumulation_steps', default=4, type=int,)

parser.add_argument('--motion_dim', type=int, default=263, help='Input motion dimension dimension')
parser.add_argument('--enc_dec_dim', type=int, default=768, help='Encoder and Decoder dimension')
parser.add_argument('--depth', type=int, default=12, help='Encoder Decoder depth')
parser.add_argument('--heads', type=int, default=10, help='Encoder Decoder number of heads')
parser.add_argument('--codebook_dim', type=int, default=128, help='codeboook dimension')
parser.add_argument('--codebook_size', type=int, default=1024, help='number of codebook embeddings')




parser.add_argument("--commit", type=float, default=1, help="hyper-parameter for the commitment loss")
parser.add_argument('--loss_vel', type=float, default=0.1, help='hyper-parameter for the velocity loss')
parser.add_argument('--recons_loss', type=str, default='l1_smooth', help='reconstruction loss')
parser.add_argument('--max_seq_length', type=int, default=200, help='max sequence length')

parser.add_argument("--num_train_iters",  default=500000,type=int)
parser.add_argument("--save_steps",  default=5000,type=int)
parser.add_argument("--logging_steps",  default=10,type=int)
parser.add_argument("--wandb_every",  default=100,type=int)
parser.add_argument("--evaluate_every",  default=10000,type=int)

## optimization
parser.add_argument('--weight-decay', default=0.0, type=float, help='weight decay')
parser.add_argument('--warmup_steps', default=4000, type=int, help='number of total iterations for warmup')
parser.add_argument('--learning_rate', default=2e-4, type=float, help='max learning rate')
parser.add_argument('--gamma', default=0.05, type=float, help="learning rate decay")
parser.add_argument('--lr_scheduler_type', default="cosine", help="learning rate schedule type")

args = parser.parse_args([])

In [7]:
from configs.config import cfg

In [9]:
argss = Dict2Class(vars(args))

In [10]:
from configs.config import cfg


In [14]:
from core.models.vqvae import VQMotionModel

model = VQMotionModel(args)

In [15]:
pkg = torch.load("/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/vq_768_128/vqvae_motion.pt", map_location = 'cpu')

In [16]:
model.load_state_dict(pkg["model"])

<All keys matched successfully>

In [26]:
model = model.cuda()

In [ ]:
!python VQ_eval.py

In [574]:
ind = model.encode(batch["motion"].cuda())
ind.shape

torch.Size([4, 177])

In [575]:
# inds = torch.randint(0,1024,(1,40))
quantized, out_motion = model.decode(ind.cuda())

In [573]:
sample_render(to_xyz(out_motion[1:2].detach().cpu()), "rnd_motion" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals/decode_test")

render start


In [584]:
batch["motion_length"]

tensor([177., 165., 114.,  74.])

In [586]:
for i in range(4):
    print(F.mse_loss(batch["motion"][i,:40,:].cpu() , out_motion[i,:40,:].cpu()))
    print(F.mse_loss(batch["motion"][i,40:int(batch["motion_length"][i]),:].cpu() , out_motion[i,40:int(batch["motion_length"][i]),:].cpu()))
    print("\n")

tensor(0.0577)
tensor(1.9838)


tensor(0.1757)
tensor(2.9448)


tensor(0.2407)
tensor(1.6494)


tensor(0.1197)
tensor(0.4883)




## Music codes to motion

In [7]:
music_paths =  glob("/srv/scratch/sanisetty3/music_motion/AIST/music/*.npy" )

In [16]:
msc = np.load(music_paths[0])

In [41]:
msc2s = msc[:,:64]

In [44]:
decoded_motion_features = model.motionDecoder(torch.Tensor(msc2s.T)[None,...])

In [45]:
decoded_motion_features.shape

torch.Size([1, 64, 263])

In [46]:
sample_render(to_xyz(decoded_motion_features.detach().cpu()), "mus_pred" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals")

render start


## VQ_eval

In [23]:
import core.models.vqvae as vqvae
import utils.utils_model as utils_model
from core.datasets import dataset_TM_eval
import utils.eval_trans as eval_trans
from core.models.evaluator_wrapper import EvaluatorModelWrapper
from core.models.vqvae import VQMotionModel
from utils.word_vectorizer import WordVectorizer
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [ ]:
dataname = "t2m"
exp_name = "vq_768_128"
out_dir = "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals"
out_dir = os.path.join(out_dir, f'{exp_name}')
os.makedirs(out_dir, exist_ok = True)

In [24]:
logger = utils_model.get_logger(out_dir)
writer = SummaryWriter(out_dir)

In [21]:


w_vectorizer = WordVectorizer('/srv/scratch/sanisetty3/music_motion/T2M-GPT/glove', 'our_vab')

checkpoint_dir = "/srv/scratch/sanisetty3/music_motion/T2M-GPT/checkpoints"
# dataset_opt_path = f'{checkpoint_dir}/kit/Comp_v6_KLD005/opt.txt' if dataname == 'kit' else f'{checkpoint_dir}/t2m/Comp_v6_KLD005/opt.txt'

eval_wrapper = EvaluatorModelWrapper(cfg.eval_model)


##### ---- Dataloader ---- #####
nb_joints = 21 if dataname == 'kit' else 22

val_loader = dataset_TM_eval.DATALoader(dataname, True, 4, w_vectorizer, unit_length=2**2)

Loading Evaluation Model Wrapper (Epoch 28) Completed!!


100%|██████████| 4384/4384 [00:11<00:00, 386.36it/s]

4648 4648
Pointer Pointing at 0


In [ ]:

fid = []
div = []
top1 = []
top2 = []
top3 = []
matching = []
repeat_time = 20
for i in tqdm(range(repeat_time)):
    best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = eval_trans.evaluation_vqvae(out_dir, val_loader, model, logger, writer, 0, best_fid=1000, best_iter=0, best_div=100, best_top1=0, best_top2=0, best_top3=0, best_matching=100, eval_wrapper=eval_wrapper, draw=False, save=False, savenpy=(i==0))
    fid.append(best_fid)
    div.append(best_div)
    top1.append(best_top1)
    top2.append(best_top2)
    top3.append(best_top3)
    matching.append(best_matching)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
print('final result:')
print('fid: ', sum(fid)/repeat_time)
print('div: ', sum(div)/repeat_time)
print('top1: ', sum(top1)/repeat_time)
print('top2: ', sum(top2)/repeat_time)
print('top3: ', sum(top3)/repeat_time)
print('matching: ', sum(matching)/repeat_time)


## T2m

In [617]:
%cd /coc/scratch/sanisetty3/music_motion/T2M-GPT


/coc/scratch/sanisetty3/music_motion/T2M-GPT


In [679]:
%cd /coc/scratch/sanisetty3/music_motion/motion_vqvae/


/coc/scratch/sanisetty3/music_motion/motion_vqvae


In [ ]:
from 

In [594]:
# !pip install git+https://github.com/openai/CLIP.git

In [662]:
import os
import json

import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import models.vqvae as vqvae
import options.option_vq as option_vq
import utils.utils_model as utils_model
from dataset import dataset_TM_eval
import utils.eval_trans as eval_trans
from utils.eval_trans import *
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper

In [605]:
# args = option_vq.get_args_parser()

In [606]:
from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')

In [596]:
dataset_opt_path = 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'
wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)


Reading checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


In [677]:
wrapper_opt.dim_movement_latent

512

In [674]:
vars(wrapper_opt)[""]

{'batch_size': 32,
 'checkpoints_dir': './checkpoints',
 'dataset_name': 't2m',
 'decomp_name': 'Decomp_SP001_SM001_H512',
 'dim_att_vec': 512,
 'dim_dec_hidden': 1024,
 'dim_movement2_dec_hidden': 512,
 'dim_movement_dec_hidden': 512,
 'dim_movement_enc_hidden': 512,
 'dim_movement_latent': 512,
 'dim_msd_hidden': 512,
 'dim_pos_hidden': 1024,
 'dim_pri_hidden': 1024,
 'dim_seq_de_hidden': 512,
 'dim_seq_en_hidden': 512,
 'dim_text_hidden': 512,
 'dim_z': 128,
 'early_stop_count': 3,
 'estimator_mod': 'bigru',
 'eval_every_e': 5,
 'feat_bias': 5,
 'fixed_steps': 5,
 'gpu_id': 1,
 'input_z': False,
 'is_continue': False,
 'is_train': False,
 'lambda_fake': 10,
 'lambda_gan_l': 0.1,
 'lambda_gan_mt': 0.1,
 'lambda_gan_mv': 0.1,
 'lambda_kld': 0.005,
 'lambda_rec': 1,
 'lambda_rec_init': 1,
 'lambda_rec_mot': 1,
 'lambda_rec_mov': 1,
 'log_every': 50,
 'lr': 0.0002,
 'max_sub_epoch': 50,
 'max_text_len': 20,
 'n_layers_dec': 1,
 'n_layers_msd': 2,
 'n_layers_pos': 1,
 'n_layers_pri': 1,


In [646]:
val_loader = dataset_TM_eval.DATALoader("t2m", True, 4, w_vectorizer, unit_length=2**2, num_workers = 0 )

100%|██████████| 4384/4384 [00:03<00:00, 1332.58it/s]

4648 4648
Pointer Pointing at 0


In [664]:
for batch in val_loader:
    break

In [665]:
word_embeddings, pos_one_hots, caption, sent_len, motion, m_length, token, name = batch

In [666]:
motion = motion.to(torch.float32)

In [661]:
m_length

tensor([192, 192, 180, 136])

In [659]:
pred_motion , ind , com_loss = model(motion.cuda())

In [ ]:
best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = eval_trans.evaluation_vqvae(args.out_dir, val_loader, net, logger, writer, 0, best_fid=1000, best_iter=0, best_div=100, best_top1=0, best_top2=0, best_top3=0, best_matching=100, eval_wrapper=eval_wrapper, draw=False, save=False, savenpy=(i==0))


In [670]:
model.eval()
nb_sample = 0

draw_org = []
draw_pred = []
draw_text = []


motion_annotation_list = []
motion_pred_list = []
R_precision_real = 0
R_precision = 0

nb_sample = 0
matching_score_real = 0
matching_score_pred = 0


word_embeddings, pos_one_hots, caption, sent_len, motion, m_length, token, name = batch
motion = motion.to(torch.float32)
motion = motion.cuda()
et, em = eval_wrapper.get_co_embeddings(word_embeddings, pos_one_hots, sent_len, motion, m_length)
bs, seq = motion.shape[0], motion.shape[1]

num_joints = 21 if motion.shape[-1] == 251 else 22

pred_pose_eval = torch.zeros((bs, seq, motion.shape[-1])).cuda()

for i in range(bs):
    pose = val_loader.dataset.inv_transform(motion[i:i+1, :m_length[i], :].detach().cpu().numpy())
    pose_xyz = recover_from_ric(torch.from_numpy(pose).float().cuda(), num_joints)


    pred_pose, ind,loss_commit = model(motion[i:i+1, :m_length[i]])
    pred_denorm = val_loader.dataset.inv_transform(pred_pose.detach().cpu().numpy())
    pred_xyz = recover_from_ric(torch.from_numpy(pred_denorm).float().cuda(), num_joints)

#     if savenpy:
#         np.save(os.path.join(out_dir, name[i]+'_gt.npy'), pose_xyz[:, :m_length[i]].cpu().numpy())
#         np.save(os.path.join(out_dir, name[i]+'_pred.npy'), pred_xyz.detach().cpu().numpy())

    pred_pose_eval[i:i+1,:m_length[i],:] = pred_pose

    if i < min(4, bs):
        draw_org.append(pose_xyz)
        draw_pred.append(pred_xyz)
        draw_text.append(caption[i])

et_pred, em_pred = eval_wrapper.get_co_embeddings(word_embeddings, pos_one_hots, sent_len, pred_pose_eval, m_length)

motion_pred_list.append(em_pred)
motion_annotation_list.append(em)

temp_R, temp_match = calculate_R_precision(et.cpu().numpy(), em.cpu().numpy(), top_k=3, sum_all=True)
R_precision_real += temp_R
matching_score_real += temp_match
temp_R, temp_match = calculate_R_precision(et_pred.cpu().numpy(), em_pred.cpu().numpy(), top_k=3, sum_all=True)
R_precision += temp_R
matching_score_pred += temp_match

nb_sample += bs

In [671]:
R_precision_real

array([3, 4, 4])

In [672]:
R_precision

array([1, 3, 4])

In [ ]:
from configs